# MSIA - Points - 3 - Surface reconstruction from point clouds

In this practical session, we will:
- load an visualize a point cloud
- design a simple RANSAC algorithm for plane fitting
- improve this approach using normal estimation

## Notebook setup

For point cloud loading we use [trimesh](https://trimesh.org/) which is one of the many python libraries that can be used for point cloud processing. As it is not available on Colab, we first install it.

In [ ]:
# install the missing package
!pip install trimesh

Then, we download the point cloud we will work with. The point cloud can be directly donwloaded from the course [github]( https://github.com/aboulch/MSIA_points) or using the following command.

In [ ]:
# load a points cloud
!wget https://github.com/aboulch/MSIA_points/releases/download/v0.0.0/mesh_a003a6585e.ply

Finally, we import the libraries useful for this practical session.
The whole practical session will be managed with numpy.

In [ ]:
import numpy as np
import tqdm
import trimesh
import math
import plotly.graph_objects as go # for visualization
from scipy.spatial import KDTree

## Loading / decimating and visualizing point clouds

In this section we define the functions needed for preparing the point cloud.

First, we give a convenient function to visualize the point clouds using the Plotly library.
It takes as input a point cloud (`pts`) and colors (`cls`, with values in $[0,1]$).
The function creates a Figure and draw the points with Scatter3D, there are several options to the function (see Plotly documentation), one of the most useful is the marker size (apparent size of the points).

In [ ]:
def point_cloud_visu(pts, cls=None):

  fig = go.Figure(
      data=[
          go.Scatter3d(
              x=pts[:,0], y=pts[:,1], z=pts[:,2],
              mode='markers',
              marker=dict(size=3, color=cls)
          )
      ],
      layout=dict(
          scene=dict(
              xaxis=dict(visible=False),
              yaxis=dict(visible=False),
              zaxis=dict(visible=False),
              aspectmode="data", #this string can be 'data', 'cube', 'auto', 'manual'
              #a custom aspectratio is defined as follows:
              aspectratio=dict(x=1, y=1, z=0.95)
          )
      )
  )
  fig.show()


### Point decimation via voxelization or random sampling

The proposed point cloud is too large to work with it efficiently.
First, we will decimate to reduce its size.
Two very common decimation procedures are *random sampling* and *voxel decimation*.

**Question:** fill `load_and_decimate_room_pc_rand` which selects `n` points randomly in the point cloud



In [ ]:
def load_and_decimate_room_pc_rand(num_pts=10000):
  """
  function: load and randomly subsample a point cloud
  input: number of points
  output: points and normals
  """

  pcd = trimesh.load("mesh_a003a6585e.ply")
  pts = pcd.vertices
  cols = pcd.visual.vertex_colors

  ### Fill this part with decimation
  ###

  # to be removed once the function is implmented
  pts = pts[:1000]
  cols = cols[:1000]

  return pts, cols[:,:3]


**Question:** fill `load_and_decimate_room_pc_voxels` which discretize the point cloud on a 3D grid of size `voxel_size` and keep one point per voxel.

In [ ]:
def load_and_decimate_room_pc_voxels(voxel_size=0.10):
  """
  function: load and subsample a point cloud (one point per voxel)
  input: number of points
  output: points and normals
  """
  # Hint: it is possible to "inflate" the coordinates of the point cloud such
  # that voxel size correspond to 1, then take integral part of the coordinates
  # and select one unique point in each voxel (np.unique)

  pcd = trimesh.load("mesh_a003a6585e.ply")
  pts = pcd.vertices
  cols = pcd.visual.vertex_colors

  ### Fill this part with decimation
  ###

  # to be removed once the function is implmented
  pts = pts[:1000]
  cols = cols[:1000]

  return pts, cols[:,:3]

We can now load the point cloud, decimate it and visualize it.
For loading we use Open3d, a 3D processing library (this library does not only load points, it contains many algorithms, beyond the scope of this course).

**Question:** Compare random decimation and voxel decimation.

In [ ]:
# load the points decimated with random sampling
pts_rand, cols_rand = load_and_decimate_room_pc_rand()

# load the points decimated using voxel grid
pts_vox, cols_vox = load_and_decimate_room_pc_voxels()

# translate one the point cloud for visu, side by side
pts_vox[:,0] += 8

# assemble the points
pts_visu = np.concatenate([pts_rand, pts_vox], axis=0)
cols_visu = np.concatenate([cols_rand, cols_vox], axis=0)
point_cloud_visu(pts_visu, cols_visu)


## Normal estimation for point cloud

In this section, we will implement a normal estimator for point clouds.
The objective is to estimate a unit vector $\mathbf{n}$ at each point $\mathbf{p}$.

To do so, we will implement the most common normal estimator: the local plane regression from [1].
As shown in the course, given a set of point $\mathcal{N}$, we compute the covariance matrix, diagonalize it and find the eigen vector corresponding to the lowest eigen vector.

**Question:** complete the function `plane_fitting_pca` to compute the regression plane for the input point cloud `pts`.

*Note:* we want it to work with batches, i.e., compute the matrices for several point clouds at once. This saves the use of slow loop later.


In [ ]:
def covariance_matrix(pts):
  # input: point cloud [N 3]
  # output: 3x3 matrix covariance matrix
  assert(len(pts.shape)==2)

  ### Fill this part
  ###

  # to be removed
  cov = np.eye(3)

  return cov

# create a random point cloud on a plane
pts = np.random.rand(1000,3)
pts[:,0] *= 2
pts[:,1] *= 0.5
pts[:,2] *= 0

# visualize the point cloud
point_cloud_visu(pts)

# compare our method with the numpy function (should give the same result)
print("Our method that works with batches:")
print(covariance_matrix(pts))
print("Numpy version, not batched")
print(np.cov(pts.T))

The plane equation is given by $ax + by + cz + d = 0$.

**Question:** complete the function to fit a plane by PCA:
The input is the point cloud `pts` (size [B,N,3]), the output is the plane parameters $(a,b,c,d)$ size [B,4].
The batch B is optional

* compute the covariance matrix
* diagonalize the matrix
* find the lowest eigen value
* find the corresponding eigen vector
* normalize it
* compute the $d$ parameter of the plane equation.


In [ ]:
def plane_fitting_pca(pts_):
  assert(len(pts.shape)==2)

  ###################
  ### Fill the part
  # note: eigen values and vectors can be computed using numpy functions
  # covariance matrix
  # diagonalize the covariance matrix
  # find the smallest eigenvalue
  # find the corresponding eigen vector (hint, use np.take_along_axis)
  # normalize it
  # compute the d parameter of the plane equation
  # create the plane equation (a b c d)
  ###################

  # to be removed
  plane = np.array([[1,0,0,0]])

  # return the plane equation
  return plane

# create an horizontal plane
pts = np.random.rand(1000,3)
pts[:,0] *= 2
pts[:,1] *= 0.5
pts[:,2] *= 0

print("Plane equation (should be [0,0,1,0] or [0,0,-1,0]):")
print(plane_fitting_pca(pts))

Now that we have plane fitting function, we can use it compute the normals in a point cloud.
To do so, for each point, we need to compute neighborhood (small point cloud) on which we will fit a plane.

For efficiency, we will use a [KDTree](https://en.wikipedia.org/wiki/K-d_tree), which is an efficient space partitioning data structure. Which allows the compute the $k$ nearest neighbors with complexity $O(k \log n)$ ($n$ being the size of the point cloud) on average.

We will use the [KDTree class from Scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.KDTree.html).

**Question:** complete the function for normal estimation.


In [ ]:
# Estimate the normals
def normal_estimation(pts, k):
  """
  input:
    - pts, point cloud (size [N, 3])
    - k, size of the neighborhoods for normal regression
  output:
    - normals (size [N,3])
  """

  ##################
  ## Fill this part
  # compute the neighbors for each plane with a KDTree
  # compute the normals by fitting a plane
  ###################

  # to be removed
  normals = np.zeros_like(pts)
  normals[:,0] = 1

  return normals


# load the point cloud
pts, cols = load_and_decimate_room_pc_voxels()

# compute the normals
normals = normal_estimation(pts, k=16)

# compute colors from normals
cols_normals = ((normals +1)/2 * 255).astype(np.uint8)

# visu
point_cloud_visu(pts, cols_normals)

## Random Sample Consensus (RANSAC)

We now enter the core of the practical session.
As seen during the course, RANSAC is a quite simple algorithm that can be summarized as:
```
INIT: points, best_hypothesis, best_inliers
REPEAT(MAX_ITER):
  GENERATE plane_hypothesis(points)
  COMPUTE inliers(points, plane_hypothesis)
  COMPARE inliers > best_inliers:
    ASSIGN best_hypothesis <- plane_hypothtesis
    ASSIGN best_inliers <- inliers
```
In this part, the proposed method is inspired from [2], which uses additional component to make it faster and more general.


### Plane hypothesis using three points

A first way to define a plane is to pick three points and compute the regression plane for these three points.

**Question:** fill `plane_from_three_points` which compute the plane equation (size [4]) associated with three points randomly selected in the point cloud.

*Note:* even if not the most efficient way to do, it is easier to reuse the function to compute normals.

Then, we need to compute the inliers for the selected plane $\mathcal{P}$. These inliers are the points of the point cloud $\mathbf{P}$ such that:
$$ \mathbf{I}(\mathbf{P}, \mathcal{P}, \delta) = \{ \mathbf{p} \in \mathbf{P} ,\: \text{s.t.} \: d(\mathbf{p}, \mathcal{P}) < \delta \}$$
 where $d$ is the Euclidean distance and $\delta$ a threshold.

**Question:** fill `get_inliers_in_plane` the function that computes the inlier given a plane and a point cloud.

In [ ]:

def plane_from_three_points(pts):
  """
  Function: compute the plane equation from 3 points randomly selected in the
  point cloud
  """
  ##################
  ### Fill this part

  ##################

  # to be removed
  plane = np.array([[1,0,0,0]])

  plane = plane[0]
  return plane

def get_inliers_in_plane(pts, plane, distance_threshold):
  """
  Function: compute the points close to a plane (up to a threshold)
  input:
    - pts
    - plane
  output:
    - mask (pts.shape[0],) of booleans: true if inliers, false if outliers
  """

  ##################
  ### Fill this part

  ##################

  # to be removed
  mask = np.ones(pts.shape[0], dtype=bool)

  return mask

# load the point cloud
pts, cols = load_and_decimate_room_pc_voxels()

# estimate a random plane
plane = plane_from_three_points(pts)

# get the inliers
mask = get_inliers_in_plane(pts, plane, 0.1)

# give red color to the inliers
cols_plane_tmp = cols[mask].copy()
cols_plane_tmp *= 0
cols_plane_tmp[:,0] = 255
cols_plane = cols.copy()
cols_plane[mask] = cols_plane_tmp

# display the point cloud
point_cloud_visu(pts,cols_plane)

Now we need to compute the maximal number of iterations (number of plane hypothesis to compute) needed to find a plane containing a minimal number of points.

As a reminder from the course.

The probablity of picking three inlier points in the point cloud is:
$$P(n) = \left( \frac{n}{N} \right)^3$$
where $n$ is the minimal number of point to be considered a valid shape, and $N$ the size of the point cloud.

Then, propability that the plane is not an inlier is:
$$ 1 - \left( \frac{n}{N} \right)^3, $$
the probability that after $s$ attempts we do not get one inlier plane is
$$ \left(1 - \left( \frac{n}{N} \right)^3 \right)^s,$$
and the probability that we get and inlier plane:
$$ P(n,s) = 1 - \left(1 - \left( \frac{n}{N} \right)^3 \right)^s.$$

We search $T$ such that the probability of success if greater than a given probability $p_t$:
$$ P(n,T) \geq p_t .$$

**Question:** fill the function `get_max_num_iter` which compute the $T$ value.

In [ ]:
def get_max_num_iter(min_pts, num_pts, proba_of_success):

  ##################
  ### Fill this part

  ##################

  # to be removed
  T = 10

  return int(T) + 1

Now we have all the ingredients to search for the best plane using the RANSAC algorithm.

**Question:** fill the function `search_one_plane` that attempts to find the best plane.

In [ ]:
def search_one_plane(pts, normals, min_pts, plane_threshold, proba_of_success):
  """
  Function: search the best plane by iteratively selecting plane hypotheses and
  retain the plane with the most inliers
  - input:
    - pts: the point cloud
    - normals: not used for this function
    - min_pts: minimal number of points in a valid shape
    - plane_threshold: the distance threshold for plane hypothesis
  - output:
    - plane_equation: equation of the plane
    - inlier_mask: boolean mask for the inliers
  """

  ##################
  ### Fill this part
  # compute max iters
  # iterate over iters
  # memorize the best plane
  ##################

  # to be removed
  cur_plane_equation = np.array([1,0,0,0])
  cur_plane_mask = np.ones(pts.shape[0], dtype=bool)

  return cur_plane_equation, cur_plane_mask

# load the points
pts, cols = load_and_decimate_room_pc_voxels()

# search for the best plane
plane_equation, plane_mask = search_one_plane(pts, None,
      min_pts=500,
      plane_threshold=0.05,
      proba_of_success=0.9)

# give red color to the inliers
cols_plane_tmp = cols[plane_mask].copy()
cols_plane_tmp *= 0
cols_plane_tmp[:,0] = 255
cols_plane = cols.copy()
cols_plane[plane_mask] = cols_plane_tmp

# display the point cloud
point_cloud_visu(pts,cols_plane)

### Plane hypothesis using one point and normal

A second way to define a plane is to pick one plane with its associated normal.

**Question:** fill `plane_from_one_point_normal` which compute the plane equation (size [4]) associated with one point randomly selected in the point cloud.


In [ ]:
def plane_from_one_point_normal(pts, normals):
  """
  Function: compute the plane equation from 3 points randomly selected in the
  point cloud
  """

  ##################
  ### Fill this part
  ##################

  # to be removed
  plane = np.array([1,0,0,0])
  return plane

Then, we need to compute the inliers for the selected plane $\mathcal{P}$.
Here, we now suppose the points to have an associated normal.
Thus, inliers should have a normal aligned with the plane normal (up to orientation).
These inliers are the points of the point cloud $\mathbf{P}$ such that:
$$ \mathbf{I}(\mathbf{P}, \mathcal{P}, \delta) = \{ \mathbf{p} \in \mathbf{P} ,\: \text{s.t.} \: d(\mathbf{p}, \mathcal{P}) < \delta \: \textbf{and} \: |<\mathbf{n}(\mathcal{P}), \mathbf{n} >| < \gamma \}$$
 where $d$ is the Euclidean distance, $\delta$ a distance threshold, $<.,.>$ is the inner product and $\gamma$ an orientation threshold.

**Question:** fill `get_inliers_in_plane_normals` this function only compute the orientation part of the inlier retrieval (we already coded the distance part in ).


In [ ]:
def get_inliers_in_plane_normals(normals, plane, orient_threshold):
  """
  Function: get the points that have the same normal as the plane (up to a threshold)
  input:
    - pts
    - plane
    - orient_threshold
  output:
    - mask (pts.shape[0],) of booleans: true if inliers, false if outliers
  """

  ##################
  ### Fill this part
  ##################

  # to be removed
  mask_normals = np.ones(normals.shape[0], dtype=bool)

  return mask_normals

# load the point cloud
pts, cols = load_and_decimate_room_pc_voxels()

# estimate the normals
normals = normal_estimation(pts, k=16)

# compute the best plane
plane = plane_from_one_point_normal(pts, normals)

# get the inliers
mask = get_inliers_in_plane_normals(normals, plane, 0.95)

# give red color to the inliers
cols_plane_tmp = cols[mask].copy()
cols_plane_tmp *= 0
cols_plane_tmp[:,0] = 255
cols_plane = cols.copy()
cols_plane[mask] = cols_plane_tmp

# display the point cloud
point_cloud_visu(pts,cols_plane)

We need also to use a different function to compute the number of iterations, as we are now picking only one point to generate the hypothesis.

**Question:** fill `get_max_num_iter_normal` to compute the number of iterations

In [ ]:
def get_max_num_iter_normal(min_pts, num_pts, proba_of_success):

  ##################
  ### Fill this part
  ##################

  # to be removed
  T = 10
  return int(T) + 1


Finally, we can use the RANSAC algorithm with this plane selector.

**Question:** fill `search_one_plane_normals`.

In [ ]:
# RANSAC loop
def search_one_plane_normals(pts, normals, min_pts, plane_threshold, orient_threshold, proba_of_success):
  """
  Function: search the best plane by iteratively selecting plane hypotheses and
  retain the plane with the most inliers
  - input:
    - pts: the point cloud
    - min_pts: minimal number of points in a valid shape
    - plane_threshold: the distance threshold for plane hypothesis
    - orient_threshold: the minimal scalar product of the normals to be
        to be considered an inlier
  - output:
    - plane_equation: equation of the plane
    - inlier_mask: boolean mask for the inliers
  """

  ##################
  ### Fill this part
  # use both masks on distance and normals
  ##################

  # to be removed
  cur_plane_equation = np.array([1,0,0,0])
  cur_plane_mask = np.ones(pts.shape[0], dtype=bool)

  return cur_plane_equation, cur_plane_mask

# load the point cloud
pts, cols = load_and_decimate_room_pc_voxels()

# estimate normals
normals = normal_estimation(pts, k=16)

# search the best plane
plane_equation, plane_mask = search_one_plane_normals(pts, normals,
  min_pts=500,
  plane_threshold=0.05,
  orient_threshold=0.95,
  proba_of_success=0.999)

# give red color to the inliers
cols_plane_tmp = cols[plane_mask].copy()
cols_plane_tmp *= 0
cols_plane_tmp[:,0] = 255
cols_plane = cols.copy()
cols_plane[plane_mask] = cols_plane_tmp

# display the point cloud
point_cloud_visu(pts,cols_plane)


### Multi-plane RANSAC

In the previous sections, we search for one plane.
In practice, we usually want to extract multiple planes in a given scene.
We now implement the function that search for planes until no plane (containing a minimal amount of points can be found).

**Question:** implement the `ransac` function.

*Note 1:* the process is iterative:
- it finds one plane
- it removes the inliers from the point cloud
- it iterate the process

*Note 2:* as the process go on the size of the point cloud decreases, thus the number of iterations needed also decreases.

In [ ]:
# RANSAC loop multi plane

def ransac(pts_, normals_, min_pts,
           plane_search_function,
           plane_search_function_args):

  pts = pts_.copy()
  normals = normals_.copy()


  ##################
  ### Fill this part
  ##################

  shapes = [
      pts[:10],
      pts[10:20],
  ]

  return shapes

# load the point cloud
pts, cols = load_and_decimate_room_pc_voxels()

# compute the normals
normals = normal_estimation(pts, k=16)

# ransac
shapes = ransac(pts, normals,
                min_pts=200,
                plane_search_function=search_one_plane_normals,
                plane_search_function_args=dict(
                    plane_threshold=0.05,
                    orient_threshold=0.9,
                    proba_of_success=0.999
                ))

# visu, with one random color for each shape
shape_colors = []
for shape_pts in shapes:
  color = np.random.randint(0,255,size=(1,3))
  color = np.repeat(color, shape_pts.shape[0], axis=0)
  shape_colors.append(color)
shapes_cat = np.concatenate(shapes, axis=0)
shape_colors_cat = np.concatenate(shape_colors, axis=0)

point_cloud_visu(shapes_cat, shape_colors_cat)

## Bibliography

[1] Hoppe, Hugues, et al. "Surface reconstruction from unorganized points." Proceedings of the 19th annual conference on computer graphics and interactive techniques. 1992.

[2] Schnabel, Ruwen, Roland Wahl, and Reinhard Klein. "Efficient RANSAC for point‐cloud shape detection." Computer graphics forum. Vol. 26. No. 2. Oxford, UK: Blackwell Publishing Ltd, 2007.